In [12]:
!pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [118]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the dataset
dataorig = pd.read_excel("C:/Users/Usuario/Documents/Projectes/ChallengeAuditoria/challenge-auditoria/data/inventory_data_month_index.xlsx")

dataorig["cost_valor"] = dataorig["preu_venda_unitari_2023"] / dataorig["cost_unitari_stock_2023"]

dataorig["proporcio_vendes_stock"] = dataorig["vendes_2023"] / dataorig["stock_final_2023"]
# Drop rows with missing values

data = dataorig.copy()
data.dropna(inplace=True)


# Encode categorical variables if needed
# (If there are any categorical variables that need encoding)

# Select relevant features for the Autoencoder
features = [ 'proporcio_variacio_preu_venda_unitari_2022_2023', 'diferencia_entrada_sortida', 'dies_ultima_sortida', 'proporcio_vendes_stock', 'cost_valor', "forecast_index"]

X = data[features]

# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and validation sets
X_train, X_val = train_test_split(X_scaled, test_size=0.2, random_state=42)

print("Shape of training data:", X_train.shape)
print("Shape of validation data:", X_val.shape)


Shape of training data: (233, 6)
Shape of validation data: (59, 6)


In [88]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

# Define Autoencoder architecture
input_dim = X_train.shape[1]
encoding_dim = 4  # Increase the size of the encoding layer

input_layer = Input(shape=(input_dim,))
encoder = Dense(128, activation='relu')(input_layer)  # Add more neurons
encoder = Dense(encoding_dim, activation='relu')(encoder)
decoder = Dense(128, activation='relu')(encoder)  # Add more neurons
decoder = Dense(input_dim, activation='relu')(decoder)

autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the Autoencoder model
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=32,
                shuffle=True,
                validation_data=(X_val, X_val))

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.9730 - val_loss: 0.9218
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1410 - val_loss: 0.9021
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9454 - val_loss: 0.8674
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0300 - val_loss: 0.8194
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8876 - val_loss: 0.7614
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7764 - val_loss: 0.7103
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7011 - val_loss: 0.6684
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7357 - val_loss: 0.6320
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.5614 - val_loss: 0.6002
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5653 - val_loss: 0.5776
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5194 - val_loss: 0.5678
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4729 - val_loss: 0.5587
Epoch 13/50

In [89]:
import tensorflow as tf

# Assuming reconstructions and X_val are numpy arrays
reconstructions = autoencoder.predict(X_val)
mse = tf.keras.losses.mean_squared_error(X_val, reconstructions)

# Calculate reconstruction errors (no need to specify axis for 1D array)
mse_numpy = mse.numpy()
reconstruction_errors = mse_numpy


# Set a threshold for anomaly detection
threshold = reconstruction_errors.mean() + 2 * reconstruction_errors.std()

# Identify assets with reconstruction errors above the threshold
anomalies_indices = [i for i, error in enumerate(reconstruction_errors) if error > threshold]
anomalies = data.iloc[anomalies_indices]

print("Assets needing depreciation:")
print(anomalies)
print(len(anomalies))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Assets needing depreciation:
    Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0   material  unitats_2022  \
38            38            38          38  133200405      376250.0   
57            57            57          57  135061014      425000.0   

    vendes_2022  preu_venda_unitari_2022  unitats_2023  vendes_2023  \
38     23075.96                 0.061331      132000.0      8650.32   
57     39559.85                 0.093082      357000.0     34693.77   

    preu_venda_unitari_2023  ...     Sep_2023     Oct_2022     Oct_2023  \
38                 0.065533  ...     0.000000  1372.369316     0.000000   
57                 0.097181  ...  4252.638944   326.701890  6157.832593   

      Nov_2022     Nov_2023     Dec_2022     Dec_2023  forecast_index  \
38  5054.69849     0.000000  4506.108044     0.000000        0.391960   
57  3340.24985  5012.315323  4232.356450  2704.797758        0.228659   

    cost_valor  proporcio_vendes_stock  
38    1.597775    

In [90]:
# Define accounting standards
salvage_value = 0  # Assumption: Salvage value is 0 (can be adjusted)
original_cost = dataorig['cost_unitari_stock_2023']  # Original cost of the asset

# Calculate the depreciation expense using the straight-line method
# We need to estimate the useful life or make an assumption
# For demonstration, let's assume a useful life of 5 years
useful_life_years = 5
depreciation_expense = (original_cost - salvage_value) / useful_life_years

# Define an acceptable range for depreciation value
# We can use a percentage of the original cost or a fixed value
# For demonstration, let's use a percentage (e.g., 10%) of the original cost
acceptable_range_percentage = 0  # 10%
acceptable_depreciation_range = original_cost * acceptable_range_percentage

# Calculate the lower and upper bounds of the acceptable range
lower_bound = original_cost - acceptable_depreciation_range
upper_bound = original_cost + acceptable_depreciation_range

# Determine assets that fall outside the acceptable range
outside_acceptable_range = (dataorig['preu_venda_unitari_2023'] < lower_bound)
assets_outside_range = dataorig[outside_acceptable_range]

print("Assets needing depreciation beyond acceptable range:")
print(assets_outside_range)


Assets needing depreciation beyond acceptable range:
     Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0   material  unitats_2022  \
1               1             1           1  115030252       75900.0   
28             28            28          28  133066532           NaN   
35             35            35          35  133100502      240000.0   
146           146           146         146  152270347           NaN   
197           197           197         197  153010114      723800.0   
198           198           198         198  153010115     7937600.0   
209           209           209         209  153110150    35825400.0   
210           210           210         210  153110151     2906800.0   
212           212           212         212  153110155     2756000.0   
213           213           213         213  153110157     7384000.0   
352           352           352         352  173430205    11835000.0   
363           363           363         363  182040131      332800.0   

     vende

We create some fake data, of values that would need depreciation:

In [91]:
data_new = {'proporcio_variacio_preu_venda_unitari_2022_2023': [-0.5, -1],
        'diferencia_entrada_sortida': [100, 30],
        'dies_ultima_sortida': [300, 500],
        'proporcio_vendes_stock' : [0.7, 3],
        'cost_valor' : [0.8, 1.1],
        'forecast_index' : [1, 0.9]}
new_dataframe = pd.DataFrame(data_new)


In [92]:
from tensorflow.keras.models import Model
encoder_model = Model(inputs=input_layer, outputs=encoder)

In [93]:
references = encoder_model.predict(new_dataframe)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


In [94]:
embeddings = encoder_model.predict(X)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [95]:
reference = references[0]

In [97]:
import numpy as np
distances = np.linalg.norm(embeddings - reference, axis=1)

print(max(distances))


1304.5515


In [119]:
print(data.iloc[241])

Unnamed: 0.2                                                              308
Unnamed: 0.1                                                              308
Unnamed: 0                                                                308
material                                                            173041357
unitats_2022                                                         896000.0
vendes_2022                                                            9290.4
preu_venda_unitari_2022                                              0.010369
unitats_2023                                                         168000.0
vendes_2023                                                           1967.28
preu_venda_unitari_2023                                               0.01171
variacio_preu_venda_unitari_2022_2023                                0.001341
proporcio_variacio_preu_venda_unitari_2022_2023                      0.129355
data_darrera_entrada                               1970-01-01 01

In [114]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
distances = scaler.fit_transform([[i] for i in distances])

In [115]:
distances = [1-d[0] for d in distances]

In [116]:
data = data.assign(autoencoder_index = distances)

AttributeError: 'dict' object has no attribute 'assign'

In [57]:
data.head()

,material,unitats_2022,vendes_2022,preu_venda_unitari_2022,unitats_2023,vendes_2023,preu_venda_unitari_2023,variacio_preu_venda_unitari_2022_2023,proporcio_variacio_preu_venda_unitari_2022_2023,data_darrera_entrada,...,Oct_2022,Oct_2023,Nov_2022,Nov_2023,Dec_2022,Dec_2023,forecast_index,cost_valor,proporcio_vendes_stock,autoencoder_index
0,114230822,1018500.0,54892.49,0.053895,885500.0,58951.87,0.066575,0.012679,0.235256,1970-01-01 01:00:45.351,...,3532.453821,2393.860683,10290.288749,3573.065705,7437.975827,4715.931899,0.210499,1.930786,0.886494,0.508620
1,115030252,75900.0,6825.62,0.089929,27600.0,2127.68,0.077090,-0.012839,-0.142771,1970-01-01 01:00:45.315,...,1047.632346,427.803972,0.000000,193.955142,287.229412,150.170606,0.480055,0.997702,0.092508,0.602062
2,115070602,884000.0,100307.84,0.113470,340000.0,34522.45,0.101537,-0.011934,-0.105171,1970-01-01 01:00:45.328,...,8875.043894,0.000000,13683.961246,2679.708550,17310.488615,2527.829452,0.393597,1.771301,0.345224,0.466443
3,115130209,23217.0,5179.48,0.223090,15600.0,3770.68,0.241710,0.018620,0.083465,1970-01-01 01:00:45.281,...,0.000000,397.675659,0.000000,466.076696,0.000000,152.991466,0.230800,1.009712,0.392779,0.478677
4,115170581,357000.0,26188.50,0.073357,1023000.0,74195.50,0.072527,-0.000830,-0.011311,1970-01-01 01:00:45.331,...,421.745776,20349.658190,3045.917856,0.000000,0.000000,496.791704,0.137538,1.423348,1.766560,0.465215


In [58]:
data = data.drop(["Unnamed: 0.2", "Unnamed: 0.1", "Unnamed: 0"], axis=1)

KeyError: "['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'] not found in axis"

In [59]:
data.to_excel("C:/Users/Usuario/Documents/Projectes/ChallengeAuditoria/challenge-auditoria/data/inventory_data_autoencoder.xlsx", index=False)